In [27]:
from pymongo import MongoClient
from dotenv import load_dotenv
import os

load_dotenv()

SSH_HOST = os.getenv("SSH_HOST")
SSH_PORT = int(os.getenv("SSH_PORT", 22))
SSH_USER = os.getenv("SSH_USER")
SSH_KEY = os.getenv("SSH_KEY")

# MongoDB 설정
MONGO_HOST = os.getenv("MONGO_HOST", "localhost")
MONGO_PORT = int(os.getenv("MONGO_PORT", 27017))
MONGO_USER = os.getenv("MONGO_USER")
MONGO_PASSWORD = os.getenv("MONGO_PASSWORD")
MONGO_AUTH_DB = os.getenv("MONGO_AUTH_DB", "admin")

import warnings
warnings.filterwarnings("ignore", module="paramiko")
from sshtunnel import SSHTunnelForwarder

server = SSHTunnelForwarder(
    (SSH_HOST, SSH_PORT),
    ssh_username=SSH_USER,
    ssh_pkey=SSH_KEY,
    remote_bind_address=(MONGO_HOST, MONGO_PORT)
)
server.start()

client = MongoClient(
    f"mongodb://{MONGO_USER}:{MONGO_PASSWORD}"
    f"@127.0.0.1:{server.local_bind_port}/?authSource={MONGO_AUTH_DB}"
    )

print(client.list_database_names())



['class']


In [28]:
db = client['class']
courses = db['courses']
courses.insert_one({
  "title": "Computer Science 101",
  "credit": 3
})

InsertOneResult(ObjectId('692d90719c21509614cd414a'), acknowledged=True)

In [29]:

students = db['student']
students.insert_many([
  {"name": "Alice",   "age": 22, "grade": "A", "major": "CS"},
  {"name": "Bob",     "age": 17, "grade": "B", "major": "EE"},
  {"name": "Charlie", "age": 19, "grade": "C", "major": "ME"},
  {"name": "David",   "age": 21, "grade": "B", "major": "CS"},
  {"name": "Eve",     "age": 15, "grade": "A", "major": "EE"},
  {"name": "Frank",   "age": 14, "grade": "B", "major": "CE"},
  {"name": "Grace",   "age": 23, "grade": "C", "major": "CS"}
])

InsertManyResult([ObjectId('692d90719c21509614cd414b'), ObjectId('692d90719c21509614cd414c'), ObjectId('692d90719c21509614cd414d'), ObjectId('692d90719c21509614cd414e'), ObjectId('692d90719c21509614cd414f'), ObjectId('692d90719c21509614cd4150'), ObjectId('692d90719c21509614cd4151')], acknowledged=True)

In [30]:
jipoo = list(students.find({'major': 'CS'}))
print(jipoo)


[{'_id': ObjectId('692d90719c21509614cd414b'), 'name': 'Alice', 'age': 22, 'grade': 'A', 'major': 'CS'}, {'_id': ObjectId('692d90719c21509614cd414e'), 'name': 'David', 'age': 21, 'grade': 'B', 'major': 'CS'}, {'_id': ObjectId('692d90719c21509614cd4151'), 'name': 'Grace', 'age': 23, 'grade': 'C', 'major': 'CS'}]


In [31]:
jipoop = list(students.find({'age': {'$gt': 20}}))
print(jipoop)

[{'_id': ObjectId('692d90719c21509614cd414b'), 'name': 'Alice', 'age': 22, 'grade': 'A', 'major': 'CS'}, {'_id': ObjectId('692d90719c21509614cd414e'), 'name': 'David', 'age': 21, 'grade': 'B', 'major': 'CS'}, {'_id': ObjectId('692d90719c21509614cd4151'), 'name': 'Grace', 'age': 23, 'grade': 'C', 'major': 'CS'}]


In [32]:
jipoo_disgust = list(students.find({'age': {'$gt': 20, '$lt': 22}}))
print(jipoo_disgust)

[{'_id': ObjectId('692d90719c21509614cd414e'), 'name': 'David', 'age': 21, 'grade': 'B', 'major': 'CS'}]


In [33]:
jishit = list(students.find({"name": {"$in": ["Alice", "Bob"]}}))
print(jishit)

[{'_id': ObjectId('692d90719c21509614cd414b'), 'name': 'Alice', 'age': 22, 'grade': 'A', 'major': 'CS'}, {'_id': ObjectId('692d90719c21509614cd414c'), 'name': 'Bob', 'age': 17, 'grade': 'B', 'major': 'EE'}]


In [34]:
jigarbage = list(students.find({'major': {'$nin': ['CS']}}))
print(jigarbage)

[{'_id': ObjectId('692d90719c21509614cd414c'), 'name': 'Bob', 'age': 17, 'grade': 'B', 'major': 'EE'}, {'_id': ObjectId('692d90719c21509614cd414d'), 'name': 'Charlie', 'age': 19, 'grade': 'C', 'major': 'ME'}, {'_id': ObjectId('692d90719c21509614cd414f'), 'name': 'Eve', 'age': 15, 'grade': 'A', 'major': 'EE'}, {'_id': ObjectId('692d90719c21509614cd4150'), 'name': 'Frank', 'age': 14, 'grade': 'B', 'major': 'CE'}]


In [35]:
jitrash_of_all_time = list(students.find({
    '$and':[
        {'age': {'$gt': 18}}, {'major': 'CS'}
        ]
}))
print(jitrash_of_all_time)

[{'_id': ObjectId('692d90719c21509614cd414b'), 'name': 'Alice', 'age': 22, 'grade': 'A', 'major': 'CS'}, {'_id': ObjectId('692d90719c21509614cd414e'), 'name': 'David', 'age': 21, 'grade': 'B', 'major': 'CS'}, {'_id': ObjectId('692d90719c21509614cd4151'), 'name': 'Grace', 'age': 23, 'grade': 'C', 'major': 'CS'}]
